In [1]:
import findspark

In [2]:
findspark.init('/Users/kiranrudresha/Documents/spark/spark-2.2.0-bin-hadoop2.7')

In [3]:
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder.appName('lr_ex').getOrCreate()

In [6]:
from pyspark.ml.regression import LinearRegression

In [9]:
ecom = spark.read.csv('/Users/kiranrudresha/Documents/spark/pyspark/Python-and-Spark-for-Big-Data-master/'
                 'Spark_for_Machine_Learning/Linear_Regression/Ecommerce_Customers.csv',
                  inferSchema=True,header=True)

In [10]:
ecom.count()

500

In [16]:
for i in ecom.head(1)[0]:
    print(i)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


In [17]:
ecom.describe()

DataFrame[summary: string, Email: string, Address: string, Avatar: string, Avg Session Length: string, Time on App: string, Time on Website: string, Length of Membership: string, Yearly Amount Spent: string]

In [18]:
ecom.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [19]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler


In [29]:
assembler = VectorAssembler(inputCols=['Avg Session Length','Time on App','Time on Website','Length of Membership']
                            , outputCol='features')

In [22]:
ecom.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [30]:
type(assembler)

pyspark.ml.feature.VectorAssembler

In [31]:
output= assembler.transform(ecom)

In [32]:
type(output)

pyspark.sql.dataframe.DataFrame

In [33]:
output.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent',
 'features']

In [34]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [36]:
data= output.select('features','Yearly Amount Spent')

In [44]:
train_data,test_data= data.randomSplit([0.7,0.3])

In [46]:
train_data.count()

357

In [47]:
test_data.count()

143

In [48]:
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [51]:
lr_model = lr.fit(train_data)

In [52]:
test_res = lr_model.evaluate(test_data)

In [54]:
test_res.residuals.show()
# residual = actual - expected 

+-------------------+
|          residuals|
+-------------------+
|  9.103221059898033|
|-12.839046988875111|
|  -6.42717588414672|
|-17.794964858060553|
| -7.901914154474525|
|-1.2798490243684455|
|  1.922949837374631|
| 3.2292612134079945|
| -4.423839262651427|
| -8.539730759457484|
| -5.236023417652177|
| -4.666063433666352|
| -6.411276782440723|
|  6.434534174414182|
|-1.8720020959104886|
| -7.370462941439996|
| -17.92367050489071|
| -6.822405921479856|
|-2.7430756341036044|
| 11.192755178482969|
+-------------------+
only showing top 20 rows



In [55]:
test_data.head(2)

[Row(features=DenseVector([29.5324, 10.9613, 37.4202, 4.0464]), Yearly Amount Spent=408.6403510726275),
 Row(features=DenseVector([30.3932, 11.803, 36.3158, 2.0838]), Yearly Amount Spent=319.9288698031936)]

In [56]:
test_res.rootMeanSquaredError

9.647349009771402

In [57]:
test_res.r2

0.9852908118954676

In [58]:
d

+-------+-----------------+--------------------+-----------+------------------+------------------+------------------+--------------------+-------------------+
|summary|            Email|             Address|     Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+-------+-----------------+--------------------+-----------+------------------+------------------+------------------+--------------------+-------------------+
|  count|              500|                 500|        500|               500|               500|               500|                 500|                500|
|   mean|             null|                null|       null| 33.05319351819619|12.052487937166134| 37.06044542094859|   3.533461555915055|  499.3140382585909|
| stddev|             null|                null|       null|0.9925631110845354|0.9942156084725424|1.0104889067564033|  0.9992775024112585|   79.3147815497068|
|    min|aaron04@yahoo.com|0001 Mack MillNor..

In [59]:
data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [60]:
unlabel = test_data.select('features')

In [61]:
unlabel.show()

+--------------------+
|            features|
+--------------------+
|[29.5324289670579...|
|[30.3931845423455...|
|[30.4925366965402...|
|[30.8162006488763...|
|[31.1280900496166...|
|[31.2606468698795...|
|[31.3091926408918...|
|[31.3584771924370...|
|[31.4252268808548...|
|[31.4474464941278...|
|[31.5147378578019...|
|[31.5171218025062...|
|[31.5257524169682...|
|[31.6548096756927...|
|[31.6610498227460...|
|[31.7207699002873...|
|[31.8164283341993...|
|[31.8745516945853...|
|[31.9120759292006...|
|[31.9262720263601...|
+--------------------+
only showing top 20 rows



In [62]:
predict = lr_model.transform(unlabel)

In [63]:
predict.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[29.5324289670579...|399.53713001272945|
|[30.3931845423455...| 332.7679167920687|
|[30.4925366965402...|288.89842160406124|
|[30.8162006488763...|283.88130580652955|
|[31.1280900496166...| 565.1546009015292|
|[31.2606468698795...| 422.6064802813198|
|[31.3091926408918...|  430.797768002559|
|[31.3584771924370...| 491.9466892360674|
|[31.4252268808548...| 535.1905579174133|
|[31.4474464941278...| 427.1424728546815|
|[31.5147378578019...| 495.0485114141136|
|[31.5171218025062...|280.58448408405206|
|[31.5257524169682...| 450.3769035923226|
|[31.6548096756927...|468.82888955313433|
|[31.6610498227460...| 418.2303556758113|
|[31.7207699002873...| 546.1453964194629|
|[31.8164283341993...| 519.0461620085471|
|[31.8745516945853...|399.10765016774735|
|[31.9120759292006...| 390.2777919398113|
|[31.9262720263601...|381.01217826584343|
+--------------------+------------

In [64]:
test_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[29.5324289670579...|  408.6403510726275|
|[30.3931845423455...|  319.9288698031936|
|[30.4925366965402...|  282.4712457199145|
|[30.8162006488763...|   266.086340948469|
|[31.1280900496166...|  557.2526867470547|
|[31.2606468698795...|  421.3266312569514|
|[31.3091926408918...|  432.7207178399336|
|[31.3584771924370...|  495.1759504494754|
|[31.4252268808548...|  530.7667186547619|
|[31.4474464941278...|   418.602742095224|
|[31.5147378578019...|  489.8124879964614|
|[31.5171218025062...|  275.9184206503857|
|[31.5257524169682...|  443.9656268098819|
|[31.6548096756927...|  475.2634237275485|
|[31.6610498227460...| 416.35835357990084|
|[31.7207699002873...|   538.774933478023|
|[31.8164283341993...| 501.12249150365636|
|[31.8745516945853...|  392.2852442462675|
|[31.9120759292006...|  387.5347163057077|
|[31.9262720263601...|  392.2049334443264|
+----------